## Graph and Sub-Graph Matching using Semantics and Topology-Only Isomorphisms

In [1]:
# Import TopologicPy modules. This is not needed on other computers
import sys
sys.path.append("C:/Users/sarwj/OneDrive - Cardiff University/Documents/GitHub/topologicpy/src")

In [2]:
from topologicpy.Vertex import Vertex
from topologicpy.CellComplex import CellComplex
from topologicpy.Topology import Topology
from topologicpy.Graph import Graph
from topologicpy.Plotly import Plotly
from topologicpy.Dictionary import Dictionary
print("TopologicPy Libraries imported.")

TopologicPy Libraries imported.


In [3]:
# Step 1: Base graph
topology = CellComplex.Prism()
g1 = Graph.ByTopology(topology, toExteriorTopologies=True)


verts1 = Graph.Vertices(g1)
for i, v in enumerate(verts1):
    d = Dictionary.ByKeysValues(["position", "remove", "removeColor"], [i+5, False, "blue"])
    v = Topology.SetDictionary(v, d)

verts2 = [Topology.Translate(v, 2, 0, 0) for v in verts1]
e1 = Graph.Edges(g1)
edges2 = [Topology.Translate(e, 2, 0, 0) for e in e1]
for i in range(15):
    g1 = Graph.RemoveVertex(g1, verts1[0])

g1 = Graph.RemoveIsolatedVertices(g1)
verts1 = Graph.Vertices(g1)
e1 = Graph.Edges(g1)

# Mark two random vertices for removal (last 3 vertices)
v1 = verts1[-3]
v2 = verts1[-2]
v3 = verts1[-1]
d1 = Topology.Dictionary(v1)
d1 = Dictionary.SetValuesAtKeys(d1, ["remove", "removeColor"], [True, "red"])
v1 = Topology.SetDictionary(v1, d1)
d2 = Topology.Dictionary(v2)
d2 = Dictionary.SetValuesAtKeys(d2, ["remove", "removeColor"], [True, "red"])
v2 = Topology.SetDictionary(v2, d2)
d3 = Topology.Dictionary(v3)
d3 = Dictionary.SetValuesAtKeys(d3, ["remove", "removeColor"], [True, "red"])
v3 = Topology.SetDictionary(v3, d3)

# Get the mesh data of g1 to reconstruct it later on.
mesh_data = Graph.MeshData(g1)
edges1 = mesh_data['edges']
vert_dicts1 = mesh_data['vertexDictionaries']
edge_dicts1 = mesh_data['edgeDictionaries']


g2 = Graph.ByVerticesEdges(verts2, edges2)
verts2 = Graph.Vertices(g2)
edges2 = Graph.Edges(g2)
# Step 3: Show the two graphs
p1 = Plotly.DataByGraph(g1, edgeColor="blue", edgeWidth=2, vertexSizeKey="position", vertexColor="blue", vertexBorderColor="blue")
p2 = Plotly.DataByGraph(g2, edgeColor="orange", edgeWidth=2, vertexSizeKey="position", vertexColor="orange", vertexBorderColor="orange")
figure = Plotly.FigureByData(p1+p2)
Plotly.Show(figure)

### Try with a vertex key. Should give only 1 match. Vertices to be removed are highlighted in red

In [4]:
# Step 4: Isomorphism matching with key
matches = Graph.Match(g1, g2, vertexKey="position")
print(len(matches), "match(es) found")

new_verts = [(Vertex.X(v), Vertex.Y(v), Vertex.Z(v)) for v in verts1]
new_dicts = [d for d in vert_dicts1]
for match in matches:
    for vi2 in match.keys(): # The key in a match is the vertex index in the second (larger) graph.
        vi1 = match.get(vi2) # The value in a match is the vertex index in the first (smaller) graph.
        new_verts[vi1] = ([Vertex.X(verts2[vi2])+0.025, Vertex.Y(verts2[vi2])+0.025, Vertex.Z(verts2[vi2])+0.025])
        new_dicts[vi1] = vert_dicts1[vi1]
        # Merge the dictionaries of the two vertices.
        d1 = Topology.Dictionary(verts1[vi1])
        # Transfer Dictionary values (e.g. remove) from the first graph to the second graph.
        d2 = Topology.Dictionary(verts2[vi2])
        d2 = Dictionary.SetValueAtKey(d2, "remove", Dictionary.ValueAtKey(d1, "remove"))
        verts2[vi2] = Topology.SetDictionary(verts2[vi2], d2)

    g1b = Graph.ByMeshData(vertices=new_verts, edges=edges1, vertexDictionaries=new_dicts, edgeDictionaries=edge_dicts1)
    p1b = Plotly.DataByGraph(g1b, edgeColor="blue", edgeWidth=3, vertexSize=6, vertexSizeKey="position", vertexColor="blue", vertexColorKey="removeColor", edgeDash=True)
    figure = Plotly.FigureByData(p2+p1b)
    Plotly.Show(figure)

1 match(es) found


### See which nodes in GraphB need to be removed

In [5]:
vertices = Graph.Vertices(g2)
print("The following vertices need to be removed:")
for i,v in enumerate(vertices):
    d = Topology.Dictionary(v)
    if Dictionary.ValueAtKey(d, "remove"):
        print("index:", i, ", position value:", Dictionary.ValueAtKey(d, "position"))

The following vertices need to be removed:
index: 12 , position value: 34
index: 13 , position value: 35
index: 21 , position value: 36


## Remove the vertices from g2, creating a g3 graph

In [6]:
vertices = Graph.Vertices(g2)
edges = Graph.Edges(g2)
g3 = Graph.ByVerticesEdges(vertices, edges)
vertices = Graph.Vertices(g3)
edges = Graph.Edges(g3)
for i,v in enumerate(vertices):
    d = Topology.Dictionary(v)
    if Dictionary.ValueAtKey(d, "remove"):
        g3 = Graph.RemoveVertex(g3, v)
g3 = Graph.RemoveIsolatedVertices(g3)

### Draw the result of removing the red vertices

In [7]:
p3 = Plotly.DataByGraph(g3, edgeColor="orange", vertexSizeKey="position", vertexColor="orange", vertexBorderColor="orange")
figure = Plotly.FigureByData(p1b+p3)
Plotly.Show(figure)
figure = Plotly.FigureByData(p3)
Plotly.Show(figure)

### Try without a vertex key. Pure topology matching. Should return tens of thousands of matches.
Remember the shape of the graph does not matter. It will look different than above, but its topology is the same.

In [9]:
# Step 5: Isomorphism matching without key
matches = Graph.Match(g1, g2)
print(len(matches), "matches found")

new_verts = [(Vertex.X(v), Vertex.Y(v), Vertex.Z(v)) for v in verts1]
new_dicts = [d for d in vert_dicts1]
for match in matches[:3]:
    for key in match.keys():
        vi2 = key
        vi1 = match.get(key)
        new_verts[vi1] = ([Vertex.X(verts2[vi2])+0.025, Vertex.Y(verts2[vi2])+0.025, Vertex.Z(verts2[vi2])+0.025])
        new_dicts[vi1] = vert_dicts1[vi1]

    g4 = Graph.ByMeshData(vertices=new_verts, edges=edges1, vertexDictionaries=new_dicts, edgeDictionaries=edge_dicts1)
    p4 = Plotly.DataByGraph(g4, edgeColor="blue", edgeWidth=4, vertexSizeKey="position", vertexColorKey="removeColor", vertexBorderColor="blue")
    figure = Plotly.FigureByData(p2+p4)
    Plotly.Show(figure)

31968 matches found
